Based on 4chan NovelAILeaks (naifu)[src](https://boards.4channel.org/g/thread/89095460#p89097704)

maintained by : [x1101](https://github.com/DEX-1101)

In [ ]:
#@title ### 0. Check GPU working status

!nvidia-smi

In [ ]:
#@title ### 1. Download Novel AI API backend, model
#@markdown If the download speed is too slow try restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
   https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar
  
!echo "Decompressing API Backend..."
!tar xf naifu.tar && rm naifu.tar
!mkdir -p /content/naifu/models/animefull-latest
!echo "Decompressing 7 GB Model..."
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest && rm animefull-latest.tar

!echo "Applying fixes..."
# fix cant import 'rank_zero_only')
%cd /content/naifu
!curl -Lo requirements.txt https://raw.githubusercontent.com/DEX-1101/SecretAI/main/fix/requirements.txt
!echo "Done."


In [ ]:
#@title ### 1. (backup) Download Model / Api Backend via Google Drive
#@markdown Might not work cause download limit by Google Drive 

%cd /content/
!gdown 1-6EuAEIZVi_fOp3_ArHeP8XjPNmQb2sa -O /content/naifu.tar
!gdown 1-8to6FC_U55GNJwIbWqwzHefFuspF_RA -O /content/animefull-latest.tar

!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar

!mkdir -p /content/naifu/models/animefull-latest
!echo "Decompressing 7 GB Model..."
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest

!echo "Applying fix..."
# fix cant import 'rank_zero_only')
%cd /content/naifu
!curl -Lo requirements.txt https://raw.githubusercontent.com/DEX-1101/SecretAI/main/fix/requirements.txt
!echo "Done."

In [ ]:
#@title ### 1. (backup) Run Directly via Google Drive
#@markdown You need the file `naifu.tar`` and `animefull-latest.tar` in your Goodle Drive Storage first, then the code will unpacking the file directly

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NAI Model/ # Path of source files, CHANGE THIS !!!
!echo "Decompressing API Backend..."
!tar xf naifu.tar -C /content

!mkdir -p /content/naifu/models/animefull-latest
!echo "Decompressing 7 GB Model..."
!tar xf animefull-latest.tar -C /content/naifu/models/animefull-latest

!echo "Applying fix..."
# fix cant import 'rank_zero_only')
%cd /content/naifu
!curl -Lo requirements.txt https://raw.githubusercontent.com/DEX-1101/SecretAI/main/fix/requirements.txt
!echo "Done."

In [ ]:
#@title ### 2. Install dependencies
#@markdown Wait patiently for the installation to complete

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared


In [ ]:
#@title ### 3. Start the model
#@markdown Just visit the output mapping address (end with `trycloudflare.com` / `bore.pub`).
#@markdown - Please wait until the model is loaded (`Application startup complete` appears) before accessing
#@markdown - The service provided by cloudflare occasionally has a request timeout, which can be replaced by bore tunnel

%cd /content/naifu
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. (Optional) Run with `animefull-latest` model of 7G
#@markdown uses the 4G size animefull-final-pruned model by default. If you want to use the 7G animefull-latest model, run this

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
# %env MODEL_PATH=models/animefull-latest
%env MODEL_PATH=models
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1`
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969
